https://radimrehurek.com/gensim/auto_examples/howtos/run_doc2vec_imdb.html

https://radimrehurek.com/gensim/models/doc2vec.html

## Configurations

In [ ]:
%matplotlib inline

In [ ]:
import logging
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
pip install sentence-transformers

In [ ]:
import pandas as pd
import numpy as np
import gensim
from sklearn.model_selection import train_test_split
import multiprocessing
from collections import OrderedDict

import gensim.models.doc2vec
assert gensim.models.doc2vec.FAST_VERSION > -1, "This will be painfully slow otherwise"

from gensim.models.doc2vec import Doc2Vec

from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import collections


In [ ]:
pip install testfixtures

In [ ]:
#'/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/Processed-Merged-Dataset-99-08-28.csv'

inputFileName = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/Balenced-Dataset-00-06-26.csv'


outputLabelledTTDS= '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/Balenced-Dataset-00-06-26.csv'
outputDocsFileName = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/DocsReadyForD2V-00-06-26.csv'

d2vModelFNmPos = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/d2vModelPositive-00-06-26.model'
d2vModelFNtPos = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/d2vModelPositive-00-06-26.txt'

d2vModelFNmNeg = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/d2vModelNegative-00-06-26.model'
d2vModelFNtNeg = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/d2vModelNegative-00-06-26.txt'

csvTopic8 = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/Topics_8.csv'
csvTopic20 = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/Topics_20.csv'
csvTopic40 = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/Topics_40.csv'
csvTopic80 = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/Topics_80.csv'
csvTopic50 = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/Topics_50.csv'
csvTopic60 = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/Topics_60.csv'
csvTopic65 = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/Topics_65.csv'
csvTopic70 = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/Topics_70.csv'


d2vModelFNm0 = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/d2vModel0-00-06-23.model'
d2vModelFNt0 = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/d2vModel0-00-06-23.txt'

d2vModelFNm1 = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/d2vModel1-00-06-23.model'
d2vModelFNt1 = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/d2vModel1-00-06-23.txt'

d2vModelFNm2 = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/d2vModel2-00-06-23.model'
d2vModelFNt2 = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/d2vModel2-00-06-23.txt'

d2vModelFNm3 = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/d2vModel3-00-06-23.model'
d2vModelFNt3 = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/d2vModel3-00-06-23.txt'

csvTopic8MostSimilar = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/SimilarDocsByRoberta/Topics_8_MostSimilar_Ro.csv'
csvTopic20MostSimilar = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/SimilarDocsByRoberta/Topics_20_MostSimilar_Ro.csv'
csvTopic40MostSimilar = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/SimilarDocsByRoberta/Topics_40_MostSimilar_Ro.csv'
csvTopic50MostSimilar = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/SimilarDocsByRoberta/Topics_50_MostSimilar_Ro.csv'
csvTopic60MostSimilar = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/SimilarDocsByRoberta/Topics_60_MostSimilar_Ro.csv'
csvTopic65MostSimilar = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/SimilarDocsByRoberta/Topics_65_MostSimilar_Ro.csv'
csvTopic70MostSimilar = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/SimilarDocsByRoberta/Topics_70_MostSimilar_Ro.csv'
csvTopic80MostSimilar = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/SimilarDocsByRoberta/Topics_80_MostSimilar_Ro.csv'
#SimilarDocsByRoberta-Albert





# Colab

In [ ]:
#No need in local
from google.colab import drive
drive.mount('/MYDRIVE', force_remount=True)

# Prepair Dataset (Dividing into Test and Train)

In [ ]:
dfReviews = pd.DataFrame()
dfReviews = pd.read_csv(inputFileName, encoding = 'utf-8', header = 0 )
dfReviews.head(3)

In [ ]:
inputFileName

In [ ]:
dfReviews.shape

In [ ]:
dfReviews = dfReviews.dropna()

In [ ]:
dfReviews.shape

In [ ]:
len (dfReviews.loc[dfReviews["Sentiment"] == 0])

In [ ]:
len (dfReviews.loc[dfReviews["Sentiment"] == 1])

In [ ]:
dfReviews[4] = 'train'

In [ ]:
dfReviews.columns

In [ ]:
dfReviews.shape

In [ ]:
testNumber=113320
trainNumber=453280

In [ ]:
dfReviews.head()

# Preparing test data

# Making embeddings

In [ ]:
dfPositiveReviews = dfReviews[dfReviews["Sentiment"] == 1]
print(len(dfPositiveReviews))
dfNegativeReviews = dfReviews[dfReviews["Sentiment"] == 0]
print(len(dfNegativeReviews))

allPositiveReviews = list()
for line in dfPositiveReviews["Text"]:
  allPositiveReviews.append( str(line) )


allNegativeReviews = list()
for line in dfNegativeReviews["Text"]:
  allNegativeReviews.append( str(line) )

In [ ]:
%time
#model = SentenceTransformer('bert-base-nli-mean-tokens')
positiveModel = SentenceTransformer("/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/nli-roberta-base-v2")
negativeModel = SentenceTransformer("/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/nli-roberta-base-v2")

In [ ]:
%time
positiveEmbeddings = positiveModel.encode(allPositiveReviews, convert_to_numpy=True)

In [ ]:
%time
negativeEmbeddings = negativeModel.encode(allNegativeReviews, convert_to_numpy=True)

print(positiveEmbeddings.shape)

print(negativeEmbeddings.shape)



# Save embeddings

In [ ]:
%time

import os
import csv
import pickle
embedding_positive_path = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/Positive-Embeddings-Roberta.pkl'
embedding_negative_path = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/Negative-Embeddings-Roberta.pkl'
print("Store file on disc")
with open(embedding_positive_path, "wb") as fOutPositive:
    pickle.dump({'sentences': allPositiveReviews, 'embeddings': positiveEmbeddings}, fOutPositive)

In [ ]:
%time

with open(embedding_negative_path, "wb") as fOutNegative:
    pickle.dump({'sentences': allNegativeReviews, 'embeddings': negativeEmbeddings}, fOutNegative)

# Load Pretrained embeddings

In [ ]:
print("Load positive pre-computed embeddings from disc")
with open(embedding_positive_path, "rb") as fInp:
    cache_data = pickle.load(fInp)
    positiveSentences = cache_data['sentences']
    positiveEmbeddings = cache_data['embeddings']

In [ ]:
print("Load negtive pre-computed embeddings from disc")
with open(embedding_negative_path, "rb") as fInn:
    cache_data = pickle.load(fInn)
    negativeSentences = cache_data['sentences']
    negativeEmbeddings = cache_data['embeddings']

# Find similarities

# Find the most similar docs tn = 8




In [ ]:


from sentence_transformers import SentenceTransformer, util
import torch

top_k = min(1000, len(allPositiveReviews))



dfTopic8 = pd.read_csv(csvTopic8)
print(dfTopic8.shape)
dfTopic8 = dfTopic8.dropna()
dfSimTopic8 = pd.DataFrame()
keywordsTopic8 = []

keywordsTopic8 = dfTopic8['Keywords']

dfSimTopicPos = pd.DataFrame(columns=["docid","SimilarityScore"])
totalVectorPos = np.zeros(768)

dfSimTopicNeg = pd.DataFrame(columns=["docid","SimilarityScore"])
totalVectorNeg = np.zeros(768)



for item in keywordsTopic8 :
  item = item.replace('[','')
  item = item.replace(']','')
  item = item.replace("'",'')
  item = item.replace(",",'')
  inferred_docvecPos = positiveModel.encode(item, convert_to_numpy=True)
  inferred_docvecNeg = negativeModel.encode(item, convert_to_numpy=True)


  cos_scores = util.pytorch_cos_sim(inferred_docvecPos, positiveEmbeddings)[0]
  top_results = torch.topk(cos_scores, k=top_k)
  for score, idx in zip(top_results[0], top_results[1]):
    dfSimTopicPos = dfSimTopicPos.append({"docid": idx, "SimilarityScore": score, "Text":allPositiveReviews[idx]}, ignore_index = True)


  cos_scores = util.pytorch_cos_sim(inferred_docvecNeg, negativeEmbeddings)[0]
  top_results = torch.topk(cos_scores, k=top_k)
  for score, idx in zip(top_results[0], top_results[1]):
    dfSimTopicNeg = dfSimTopicNeg.append({"docid": idx + 283300, "SimilarityScore": score, "Text":allNegativeReviews[idx]}, ignore_index = True)

  #end of for




print("Number of unique Positive documents", dfSimTopicPos["docid"].nunique())
print("Number of unique Negative documents", dfSimTopicNeg["docid"].nunique())

dfSimTopicPos.sort_values(by='SimilarityScore', ascending=False, inplace=True)
dfSimTopicNeg.sort_values(by='SimilarityScore', ascending=False, inplace=True)

dfSimTopicPos.drop_duplicates(subset=["docid"], keep='first', inplace = True)
dfSimTopicNeg.drop_duplicates(subset=["docid"], keep='first', inplace = True)

print("Number of unique Positive documents after droping duplicates", dfSimTopicPos["docid"].nunique())
print("Number of unique Negative documents after droping duplicates", dfSimTopicNeg["docid"].nunique())

#Sampling 1000 positive and 1000 negatives
dfSimTopicPos.sort_values(by='SimilarityScore', ascending=False, inplace=True)
dfSimTopicNeg.sort_values(by='SimilarityScore', ascending=False, inplace=True)

dfSimTopicPos = dfSimTopicPos.iloc[0:1000, :]
dfSimTopicNeg = dfSimTopicNeg.iloc[0:1000, :]

#end Sampling 1000 positive and 1000 negatives


print("Max Positive:" , max(dfSimTopicPos["SimilarityScore"]))
print("Max Negatives:" , max(dfSimTopicNeg["SimilarityScore"]))

print("Min Positive:" , min(dfSimTopicPos["SimilarityScore"]))
print("Min Negatives:" , min(dfSimTopicNeg["SimilarityScore"]))

print("Mean Positive:", dfSimTopicPos["SimilarityScore"].mean())
print("Mean Negative:", dfSimTopicNeg["SimilarityScore"].mean())

print("Std Positive:",dfSimTopicPos["SimilarityScore"].std())
print("Std Negative:",dfSimTopicNeg["SimilarityScore"].std())


print("len Positive:" , len(dfSimTopicPos))
print("len Negatives:" , len(dfSimTopicNeg))

dfAll8 = dfSimTopicNeg.append(dfSimTopicPos)
dfAll8.to_csv(csvTopic8MostSimilar, header=True)



# Find the most similar docs tn = 20

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch

top_k = min(1000, len(allPositiveReviews))

dfTopic20 = pd.read_csv(csvTopic20)
print(dfTopic20.shape)
dfTopic20 = dfTopic20.dropna()
dfSimTopic20 = pd.DataFrame()
keywordsTopic20 = []

keywordsTopic20 = dfTopic20['Keywords']

dfSimTopicPos = pd.DataFrame(columns=["docid","SimilarityScore"])
totalVectorPos = np.zeros(768)

dfSimTopicNeg = pd.DataFrame(columns=["docid","SimilarityScore"])
totalVectorNeg = np.zeros(768)



for item in keywordsTopic20 :
  item = item.replace('[','')
  item = item.replace(']','')
  item = item.replace("'",'')
  item = item.replace(",",'')
  inferred_docvecPos = positiveModel.encode(item, convert_to_numpy=True)
  inferred_docvecNeg = negativeModel.encode(item, convert_to_numpy=True)


  cos_scores = util.pytorch_cos_sim(inferred_docvecPos, positiveEmbeddings)[0]
  top_results = torch.topk(cos_scores, k=top_k)
  for score, idx in zip(top_results[0], top_results[1]):
    dfSimTopicPos = dfSimTopicPos.append({"docid": idx, "SimilarityScore": score, "Text":allPositiveReviews[idx]}, ignore_index = True)


  cos_scores = util.pytorch_cos_sim(inferred_docvecNeg, negativeEmbeddings)[0]
  top_results = torch.topk(cos_scores, k=top_k)
  for score, idx in zip(top_results[0], top_results[1]):
    dfSimTopicNeg = dfSimTopicNeg.append({"docid": idx + 283300, "SimilarityScore": score, "Text":allNegativeReviews[idx]}, ignore_index = True)



  #end of for




print("Number of unique Positive documents", dfSimTopicPos["docid"].nunique())
print("Number of unique Negative documents", dfSimTopicNeg["docid"].nunique())

dfSimTopicPos.sort_values(by='SimilarityScore', ascending=False, inplace=True)
dfSimTopicNeg.sort_values(by='SimilarityScore', ascending=False, inplace=True)

dfSimTopicPos.drop_duplicates(subset=["docid"], keep='first', inplace = True)
dfSimTopicNeg.drop_duplicates(subset=["docid"], keep='first', inplace = True)

print("Number of unique Positive documents after droping duplicates", dfSimTopicPos["docid"].nunique())
print("Number of unique Negative documents after droping duplicates", dfSimTopicNeg["docid"].nunique())

#Sampling 1000 positive and 1000 negatives
dfSimTopicPos.sort_values(by='SimilarityScore', ascending=False, inplace=True)
dfSimTopicNeg.sort_values(by='SimilarityScore', ascending=False, inplace=True)

dfSimTopicPos = dfSimTopicPos.iloc[0:1000, :]
dfSimTopicNeg = dfSimTopicNeg.iloc[0:1000, :]

#end Sampling 1000 positive and 1000 negatives


print("Max Positive:" , max(dfSimTopicPos["SimilarityScore"]))
print("Max Negatives:" , max(dfSimTopicNeg["SimilarityScore"]))

print("Min Positive:" , min(dfSimTopicPos["SimilarityScore"]))
print("Min Negatives:" , min(dfSimTopicNeg["SimilarityScore"]))

print("Mean Positive:", dfSimTopicPos["SimilarityScore"].mean())
print("Mean Positive:", dfSimTopicNeg["SimilarityScore"].mean())

print("Std Positive:",dfSimTopicPos["SimilarityScore"].std())
print("Std Negative:",dfSimTopicNeg["SimilarityScore"].std())


print("len Positive:" , len(dfSimTopicPos))
print("len Negatives:" , len(dfSimTopicNeg))

dfAll20 = dfSimTopicNeg.append(dfSimTopicPos)
dfAll20.to_csv(csvTopic20MostSimilar, header=True)



# Find the most similar docs tn = 40

In [ ]:

from sentence_transformers import SentenceTransformer, util
import torch

top_k = min(1000, len(allPositiveReviews))
dfTopic40 = pd.read_csv(csvTopic40)
print(dfTopic40.shape)
dfTopic40 = dfTopic40.dropna()
dfSimTopic40 = pd.DataFrame()
keywordsTopic40 = []

keywordsTopic40 = dfTopic40['Keywords']

dfSimTopicPos = pd.DataFrame(columns=["docid","SimilarityScore"])
totalVectorPos = np.zeros(768)

dfSimTopicNeg = pd.DataFrame(columns=["docid","SimilarityScore"])
totalVectorNeg = np.zeros(768)



for item in keywordsTopic40 :
  item = item.replace('[','')
  item = item.replace(']','')
  item = item.replace("'",'')
  item = item.replace(",",'')
  inferred_docvecPos = positiveModel.encode(item, convert_to_numpy=True)
  inferred_docvecNeg = negativeModel.encode(item, convert_to_numpy=True)


  cos_scores = util.pytorch_cos_sim(inferred_docvecPos, positiveEmbeddings)[0]
  top_results = torch.topk(cos_scores, k=top_k)
  for score, idx in zip(top_results[0], top_results[1]):
    dfSimTopicPos = dfSimTopicPos.append({"docid": idx, "SimilarityScore": score, "Text":allPositiveReviews[idx]}, ignore_index = True)


  cos_scores = util.pytorch_cos_sim(inferred_docvecNeg, negativeEmbeddings)[0]
  top_results = torch.topk(cos_scores, k=top_k)
  for score, idx in zip(top_results[0], top_results[1]):
    dfSimTopicNeg = dfSimTopicNeg.append({"docid": idx + 283300, "SimilarityScore": score, "Text":allNegativeReviews[idx]}, ignore_index = True)


  #end of for




print("Number of unique Positive documents", dfSimTopicPos["docid"].nunique())
print("Number of unique Negative documents", dfSimTopicNeg["docid"].nunique())

dfSimTopicPos.sort_values(by='SimilarityScore', ascending=False, inplace=True)
dfSimTopicNeg.sort_values(by='SimilarityScore', ascending=False, inplace=True)

dfSimTopicPos.drop_duplicates(subset=["docid"], keep='first', inplace = True)
dfSimTopicNeg.drop_duplicates(subset=["docid"], keep='first', inplace = True)

print("Number of unique Positive documents after droping duplicates", dfSimTopicPos["docid"].nunique())
print("Number of unique Negative documents after droping duplicates", dfSimTopicNeg["docid"].nunique())

#Sampling 1000 positive and 1000 negatives
dfSimTopicPos.sort_values(by='SimilarityScore', ascending=False, inplace=True)
dfSimTopicNeg.sort_values(by='SimilarityScore', ascending=False, inplace=True)

dfSimTopicPos = dfSimTopicPos.iloc[0:1000, :]
dfSimTopicNeg = dfSimTopicNeg.iloc[0:1000, :]

#end Sampling 1000 positive and 1000 negatives


print("Max Positive:" , max(dfSimTopicPos["SimilarityScore"]))
print("Max Negatives:" , max(dfSimTopicNeg["SimilarityScore"]))

print("Min Positive:" , min(dfSimTopicPos["SimilarityScore"]))
print("Min Negatives:" , min(dfSimTopicNeg["SimilarityScore"]))

print("Mean Positive:", dfSimTopicPos["SimilarityScore"].mean())
print("Mean Positive:", dfSimTopicNeg["SimilarityScore"].mean())

print("Std Positive:",dfSimTopicPos["SimilarityScore"].std())
print("Std Negative:",dfSimTopicNeg["SimilarityScore"].std())


print("len Positive:" , len(dfSimTopicPos))
print("len Negatives:" , len(dfSimTopicNeg))

dfAll40 = dfSimTopicNeg.append(dfSimTopicPos)
dfAll40.to_csv(csvTopic40MostSimilar, header=True)



# Find the most similar docs tn = 50

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch

top_k = min(1000, len(allPositiveReviews))

dfTopic50 = pd.read_csv(csvTopic50)
print(dfTopic50.shape)
dfTopic50 = dfTopic50.dropna()
dfSimTopic50 = pd.DataFrame()
keywordsTopic50 = []

keywordsTopic50 = dfTopic50['Keywords']

dfSimTopicPos = pd.DataFrame(columns=["docid","SimilarityScore"])
totalVectorPos = np.zeros(768)

dfSimTopicNeg = pd.DataFrame(columns=["docid","SimilarityScore"])
totalVectorNeg = np.zeros(768)



for item in keywordsTopic50 :
  item = item.replace('[','')
  item = item.replace(']','')
  item = item.replace("'",'')
  item = item.replace(",",'')
  inferred_docvecPos = positiveModel.encode(item, convert_to_numpy=True)
  inferred_docvecNeg = negativeModel.encode(item, convert_to_numpy=True)


  cos_scores = util.pytorch_cos_sim(inferred_docvecPos, positiveEmbeddings)[0]
  top_results = torch.topk(cos_scores, k=top_k)
  for score, idx in zip(top_results[0], top_results[1]):
    dfSimTopicPos = dfSimTopicPos.append({"docid": idx, "SimilarityScore": score, "Text":allPositiveReviews[idx]}, ignore_index = True)


  cos_scores = util.pytorch_cos_sim(inferred_docvecNeg, negativeEmbeddings)[0]
  top_results = torch.topk(cos_scores, k=top_k)
  for score, idx in zip(top_results[0], top_results[1]):
    dfSimTopicNeg = dfSimTopicNeg.append({"docid": idx + 283300, "SimilarityScore": score, "Text":allNegativeReviews[idx]}, ignore_index = True)

  #end of for




print("Number of unique Positive documents", dfSimTopicPos["docid"].nunique())
print("Number of unique Negative documents", dfSimTopicNeg["docid"].nunique())

dfSimTopicPos.sort_values(by='SimilarityScore', ascending=False, inplace=True)
dfSimTopicNeg.sort_values(by='SimilarityScore', ascending=False, inplace=True)

dfSimTopicPos.drop_duplicates(subset=["docid"], keep='first', inplace = True)
dfSimTopicNeg.drop_duplicates(subset=["docid"], keep='first', inplace = True)

print("Number of unique Positive documents after droping duplicates", dfSimTopicPos["docid"].nunique())
print("Number of unique Negative documents after droping duplicates", dfSimTopicNeg["docid"].nunique())

#Sampling 1000 positive and 1000 negatives
dfSimTopicPos.sort_values(by='SimilarityScore', ascending=False, inplace=True)
dfSimTopicNeg.sort_values(by='SimilarityScore', ascending=False, inplace=True)

dfSimTopicPos = dfSimTopicPos.iloc[0:1000, :]
dfSimTopicNeg = dfSimTopicNeg.iloc[0:1000, :]

#end Sampling 1000 positive and 1000 negatives


print("Max Positive:" , max(dfSimTopicPos["SimilarityScore"]))
print("Max Negatives:" , max(dfSimTopicNeg["SimilarityScore"]))

print("Min Positive:" , min(dfSimTopicPos["SimilarityScore"]))
print("Min Negatives:" , min(dfSimTopicNeg["SimilarityScore"]))

print("Mean Positive:", dfSimTopicPos["SimilarityScore"].mean())
print("Mean Positive:", dfSimTopicNeg["SimilarityScore"].mean())

print("Std Positive:",dfSimTopicPos["SimilarityScore"].std())
print("Std Negative:",dfSimTopicNeg["SimilarityScore"].std())


print("len Positive:" , len(dfSimTopicPos))
print("len Negatives:" , len(dfSimTopicNeg))

dfAll50 = dfSimTopicNeg.append(dfSimTopicPos)
dfAll50.to_csv(csvTopic50MostSimilar, header=True)



# Find the most similar docs tn = 60

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch

top_k = min(1000, len(allPositiveReviews))


dfTopic60 = pd.read_csv(csvTopic60)
print(dfTopic60.shape)
dfTopic60 = dfTopic60.dropna()
dfSimTopic60 = pd.DataFrame()
keywordsTopic60 = []

keywordsTopic60 = dfTopic60['Keywords']

dfSimTopicPos = pd.DataFrame(columns=["docid","SimilarityScore"])
totalVectorPos = np.zeros(768)

dfSimTopicNeg = pd.DataFrame(columns=["docid","SimilarityScore"])
totalVectorNeg = np.zeros(768)



for item in keywordsTopic60 :
  item = item.replace('[','')
  item = item.replace(']','')
  item = item.replace("'",'')
  item = item.replace(",",'')
  inferred_docvecPos = positiveModel.encode(item, convert_to_numpy=True)
  inferred_docvecNeg = negativeModel.encode(item, convert_to_numpy=True)


  cos_scores = util.pytorch_cos_sim(inferred_docvecPos, positiveEmbeddings)[0]
  top_results = torch.topk(cos_scores, k=top_k)
  for score, idx in zip(top_results[0], top_results[1]):
    dfSimTopicPos = dfSimTopicPos.append({"docid": idx, "SimilarityScore": score, "Text":allPositiveReviews[idx]}, ignore_index = True)


  cos_scores = util.pytorch_cos_sim(inferred_docvecNeg, negativeEmbeddings)[0]
  top_results = torch.topk(cos_scores, k=top_k)
  for score, idx in zip(top_results[0], top_results[1]):
    dfSimTopicNeg = dfSimTopicNeg.append({"docid": idx + 283300, "SimilarityScore": score, "Text":allNegativeReviews[idx]}, ignore_index = True)



  #end of for




print("Number of unique Positive documents", dfSimTopicPos["docid"].nunique())
print("Number of unique Negative documents", dfSimTopicNeg["docid"].nunique())

dfSimTopicPos.sort_values(by='SimilarityScore', ascending=False, inplace=True)
dfSimTopicNeg.sort_values(by='SimilarityScore', ascending=False, inplace=True)

dfSimTopicPos.drop_duplicates(subset=["docid"], keep='first', inplace = True)
dfSimTopicNeg.drop_duplicates(subset=["docid"], keep='first', inplace = True)

print("Number of unique Positive documents after droping duplicates", dfSimTopicPos["docid"].nunique())
print("Number of unique Negative documents after droping duplicates", dfSimTopicNeg["docid"].nunique())

#Sampling 1000 positive and 1000 negatives
dfSimTopicPos.sort_values(by='SimilarityScore', ascending=False, inplace=True)
dfSimTopicNeg.sort_values(by='SimilarityScore', ascending=False, inplace=True)

dfSimTopicPos = dfSimTopicPos.iloc[0:1000, :]
dfSimTopicNeg = dfSimTopicNeg.iloc[0:1000, :]

#end Sampling 1000 positive and 1000 negatives


print("Max Positive:" , max(dfSimTopicPos["SimilarityScore"]))
print("Max Negatives:" , max(dfSimTopicNeg["SimilarityScore"]))

print("Min Positive:" , min(dfSimTopicPos["SimilarityScore"]))
print("Min Negatives:" , min(dfSimTopicNeg["SimilarityScore"]))

print("Mean Positive:", dfSimTopicPos["SimilarityScore"].mean())
print("Mean Positive:", dfSimTopicNeg["SimilarityScore"].mean())

print("Std Positive:",dfSimTopicPos["SimilarityScore"].std())
print("Std Negative:",dfSimTopicNeg["SimilarityScore"].std())


print("len Positive:" , len(dfSimTopicPos))
print("len Negatives:" , len(dfSimTopicNeg))

dfAll60 = dfSimTopicNeg.append(dfSimTopicPos)
dfAll60.to_csv(csvTopic60MostSimilar, header=True)



# Find the most similar docs tn = 65

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch

top_k = min(1000, len(allPositiveReviews))




dfTopic65 = pd.read_csv(csvTopic65)
print(dfTopic65.shape)
dfTopic65 = dfTopic65.dropna()
dfSimTopic65 = pd.DataFrame()
keywordsTopic65 = []

keywordsTopic65 = dfTopic65['Keywords']

dfSimTopicPos = pd.DataFrame(columns=["docid","SimilarityScore"])
totalVectorPos = np.zeros(768)

dfSimTopicNeg = pd.DataFrame(columns=["docid","SimilarityScore"])
totalVectorNeg = np.zeros(768)



for item in keywordsTopic65 :
  item = item.replace('[','')
  item = item.replace(']','')
  item = item.replace("'",'')
  item = item.replace(",",'')
  inferred_docvecPos = positiveModel.encode(item, convert_to_numpy=True)
  inferred_docvecNeg = negativeModel.encode(item, convert_to_numpy=True)


  cos_scores = util.pytorch_cos_sim(inferred_docvecPos, positiveEmbeddings)[0]
  top_results = torch.topk(cos_scores, k=top_k)
  for score, idx in zip(top_results[0], top_results[1]):
    dfSimTopicPos = dfSimTopicPos.append({"docid": idx, "SimilarityScore": score, "Text":allPositiveReviews[idx]}, ignore_index = True)


  cos_scores = util.pytorch_cos_sim(inferred_docvecNeg, negativeEmbeddings)[0]
  top_results = torch.topk(cos_scores, k=top_k)
  for score, idx in zip(top_results[0], top_results[1]):
    dfSimTopicNeg = dfSimTopicNeg.append({"docid": idx + 283300, "SimilarityScore": score, "Text":allNegativeReviews[idx]}, ignore_index = True)



  #end of for




print("Number of unique Positive documents", dfSimTopicPos["docid"].nunique())
print("Number of unique Negative documents", dfSimTopicNeg["docid"].nunique())

dfSimTopicPos.sort_values(by='SimilarityScore', ascending=False, inplace=True)
dfSimTopicNeg.sort_values(by='SimilarityScore', ascending=False, inplace=True)

dfSimTopicPos.drop_duplicates(subset=["docid"], keep='first', inplace = True)
dfSimTopicNeg.drop_duplicates(subset=["docid"], keep='first', inplace = True)

print("Number of unique Positive documents after droping duplicates", dfSimTopicPos["docid"].nunique())
print("Number of unique Negative documents after droping duplicates", dfSimTopicNeg["docid"].nunique())

#Sampling 1000 positive and 1000 negatives
dfSimTopicPos.sort_values(by='SimilarityScore', ascending=False, inplace=True)
dfSimTopicNeg.sort_values(by='SimilarityScore', ascending=False, inplace=True)

dfSimTopicPos = dfSimTopicPos.iloc[0:1000, :]
dfSimTopicNeg = dfSimTopicNeg.iloc[0:1000, :]

#end Sampling 1000 positive and 1000 negatives


print("Max Positive:" , max(dfSimTopicPos["SimilarityScore"]))
print("Max Negatives:" , max(dfSimTopicNeg["SimilarityScore"]))

print("Min Positive:" , min(dfSimTopicPos["SimilarityScore"]))
print("Min Negatives:" , min(dfSimTopicNeg["SimilarityScore"]))

print("Mean Positive:", dfSimTopicPos["SimilarityScore"].mean())
print("Mean Positive:", dfSimTopicNeg["SimilarityScore"].mean())

print("Std Positive:",dfSimTopicPos["SimilarityScore"].std())
print("Std Negative:",dfSimTopicNeg["SimilarityScore"].std())


print("len Positive:" , len(dfSimTopicPos))
print("len Negatives:" , len(dfSimTopicNeg))

dfAll65 = dfSimTopicNeg.append(dfSimTopicPos)
dfAll65.to_csv(csvTopic65MostSimilar, header=True)



# Find the most similar docs tn = 70

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch

top_k = min(500, len(allPositiveReviews))

dfTopic70 = pd.read_csv(csvTopic70)
print(dfTopic70.shape)
dfTopic70 = dfTopic70.dropna()
dfSimTopic70 = pd.DataFrame()
keywordsTopic70 = []

keywordsTopic70 = dfTopic70['Keywords']

dfSimTopicPos = pd.DataFrame(columns=["docid","SimilarityScore"])
totalVectorPos = np.zeros(768)

dfSimTopicNeg = pd.DataFrame(columns=["docid","SimilarityScore"])
totalVectorNeg = np.zeros(768)



for item in keywordsTopic70 :
  item = item.replace('[','')
  item = item.replace(']','')
  item = item.replace("'",'')
  item = item.replace(",",'')
  inferred_docvecPos = positiveModel.encode(item, convert_to_numpy=True)
  inferred_docvecNeg = negativeModel.encode(item, convert_to_numpy=True)


  cos_scores = util.pytorch_cos_sim(inferred_docvecPos, positiveEmbeddings)[0]
  top_results = torch.topk(cos_scores, k=top_k)
  for score, idx in zip(top_results[0], top_results[1]):
    dfSimTopicPos = dfSimTopicPos.append({"docid": idx, "SimilarityScore": score, "Text":allPositiveReviews[idx]}, ignore_index = True)


  cos_scores = util.pytorch_cos_sim(inferred_docvecNeg, negativeEmbeddings)[0]
  top_results = torch.topk(cos_scores, k=top_k)
  for score, idx in zip(top_results[0], top_results[1]):
    dfSimTopicNeg = dfSimTopicNeg.append({"docid": idx + 283300, "SimilarityScore": score, "Text":allNegativeReviews[idx]}, ignore_index = True)



  #end of for




print("Number of unique Positive documents", dfSimTopicPos["docid"].nunique())
print("Number of unique Negative documents", dfSimTopicNeg["docid"].nunique())

dfSimTopicPos.sort_values(by='SimilarityScore', ascending=False, inplace=True)
dfSimTopicNeg.sort_values(by='SimilarityScore', ascending=False, inplace=True)

dfSimTopicPos.drop_duplicates(subset=["docid"], keep='first', inplace = True)
dfSimTopicNeg.drop_duplicates(subset=["docid"], keep='first', inplace = True)

print("Number of unique Positive documents after droping duplicates", dfSimTopicPos["docid"].nunique())
print("Number of unique Negative documents after droping duplicates", dfSimTopicNeg["docid"].nunique())

#Sampling 1000 positive and 1000 negatives
dfSimTopicPos.sort_values(by='SimilarityScore', ascending=False, inplace=True)
dfSimTopicNeg.sort_values(by='SimilarityScore', ascending=False, inplace=True)

dfSimTopicPos = dfSimTopicPos.iloc[0:1000, :]
dfSimTopicNeg = dfSimTopicNeg.iloc[0:1000, :]

#end Sampling 1000 positive and 1000 negatives


print("Max Positive:" , max(dfSimTopicPos["SimilarityScore"]))
print("Max Negatives:" , max(dfSimTopicNeg["SimilarityScore"]))

print("Min Positive:" , min(dfSimTopicPos["SimilarityScore"]))
print("Min Negatives:" , min(dfSimTopicNeg["SimilarityScore"]))

print("Mean Positive:", dfSimTopicPos["SimilarityScore"].mean())
print("Mean Positive:", dfSimTopicNeg["SimilarityScore"].mean())

print("Std Positive:",dfSimTopicPos["SimilarityScore"].std())
print("Std Negative:",dfSimTopicNeg["SimilarityScore"].std())


print("len Positive:" , len(dfSimTopicPos))
print("len Negatives:" , len(dfSimTopicNeg))

dfAll70 = dfSimTopicNeg.append(dfSimTopicPos)
dfAll70.to_csv(csvTopic70MostSimilar, header=True)



# Find the most similar docs tn = 80

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch

top_k = min(500, len(allPositiveReviews))



dfTopic80 = pd.read_csv(csvTopic80)
print(dfTopic80.shape)
dfTopic80 = dfTopic80.dropna()
dfSimTopic80 = pd.DataFrame()
keywordsTopic80 = []

keywordsTopic80 = dfTopic80['Keywords']

dfSimTopicPos = pd.DataFrame(columns=["docid","SimilarityScore"])
totalVectorPos = np.zeros(768)

dfSimTopicNeg = pd.DataFrame(columns=["docid","SimilarityScore"])
totalVectorNeg = np.zeros(768)



for item in keywordsTopic80 :
  item = item.replace('[','')
  item = item.replace(']','')
  item = item.replace("'",'')
  item = item.replace(",",'')
  inferred_docvecPos = positiveModel.encode(item, convert_to_numpy=True)
  inferred_docvecNeg = negativeModel.encode(item, convert_to_numpy=True)


  cos_scores = util.pytorch_cos_sim(inferred_docvecPos, positiveEmbeddings)[0]
  top_results = torch.topk(cos_scores, k=top_k)
  for score, idx in zip(top_results[0], top_results[1]):
    dfSimTopicPos = dfSimTopicPos.append({"docid": idx, "SimilarityScore": score, "Text":allPositiveReviews[idx]}, ignore_index = True)


  cos_scores = util.pytorch_cos_sim(inferred_docvecNeg, negativeEmbeddings)[0]
  top_results = torch.topk(cos_scores, k=top_k)
  for score, idx in zip(top_results[0], top_results[1]):
    dfSimTopicNeg = dfSimTopicNeg.append({"docid": idx + 283300, "SimilarityScore": score, "Text":allNegativeReviews[idx]}, ignore_index = True)



print("end of for")




print("Number of unique Positive documents", dfSimTopicPos["docid"].nunique())
print("Number of unique Negative documents", dfSimTopicNeg["docid"].nunique())

dfSimTopicPos.sort_values(by='SimilarityScore', ascending=False, inplace=True)
dfSimTopicNeg.sort_values(by='SimilarityScore', ascending=False, inplace=True)

dfSimTopicPos.drop_duplicates(subset=["docid"], keep='first', inplace = True)
dfSimTopicNeg.drop_duplicates(subset=["docid"], keep='first', inplace = True)

print("Number of unique Positive documents after droping duplicates", dfSimTopicPos["docid"].nunique())
print("Number of unique Negative documents after droping duplicates", dfSimTopicNeg["docid"].nunique())

#Sampling 1000 positive and 1000 negatives
dfSimTopicPos.sort_values(by='SimilarityScore', ascending=False, inplace=True)
dfSimTopicNeg.sort_values(by='SimilarityScore', ascending=False, inplace=True)

dfSimTopicPos = dfSimTopicPos.iloc[0:1000, :]
dfSimTopicNeg = dfSimTopicNeg.iloc[0:1000, :]

#end Sampling 1000 positive and 1000 negatives


print("Max Positive:" , max(dfSimTopicPos["SimilarityScore"]))
print("Max Negatives:" , max(dfSimTopicNeg["SimilarityScore"]))

print("Min Positive:" , min(dfSimTopicPos["SimilarityScore"]))
print("Min Negatives:" , min(dfSimTopicNeg["SimilarityScore"]))

print("Mean Positive:", dfSimTopicPos["SimilarityScore"].mean())
print("Mean Positive:", dfSimTopicNeg["SimilarityScore"].mean())

print("Std Positive:",dfSimTopicPos["SimilarityScore"].std())
print("Std Negative:",dfSimTopicNeg["SimilarityScore"].std())


print("len Positive:" , len(dfSimTopicPos))
print("len Negatives:" , len(dfSimTopicNeg))

dfAll80 = dfSimTopicNeg.append(dfSimTopicPos)
dfAll80.to_csv(csvTopic80MostSimilar, header=True)

